In [ ]:
import sys
sys.path.append('../')

### Run the optimization

In [ ]:
import os

import numpy as np
from common import get_dataframe, evaluate_best_coin, objective

import mlflow
import optuna

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM_RNN_SMA')

df = get_dataframe(add_sma_columns=True)
criptos = df.filter(like='close', axis=1).columns
output_shape = 1


for cripto in criptos:

    coin = cripto.split('_')[1]
    cripto_data = df.filter(like=coin, axis=1)

    data = np.array(cripto_data)

    # Create an Optuna study
    study_name = coin  # Unique identifier of the study.
    storage_name = "sqlite:///training_sma.db"
    pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=10)
    study = optuna.create_study(study_name=coin, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, coin, output_shape), n_trials=75)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(coin, data, best_params, output_shape)
    except ValueError:
        print("No best hyperparameters found")